In [1]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}

In [1]:
%run __init__.ipynb

Successfully connected to MongoDB


In [4]:
hc = bob.HealthCheck()
hc_key = "99c0c879-0fe1-45c5-b5a6-95f0c2b1b203"
if use_healthchecks:
    step_1 = hc.start(hc_key)
    print('Start ==>', step_1)

HealthCheck init in prod mode
Start ==> Start ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-04-22


In [3]:
#--- Covid-19 France data updated everyday at 19:00PM
%run "covid-19_france.ipynb"

Script execution completed at 21/04/2020 20:44:44. Time: --- 0.9332618713378906 secnds ---
Script execution completed at 21/04/2020 20:44:44. Time: --- 0.010567903518676758 secnds ---
Dataframe FRANCE_DB_CONCAT successfully save in database covid-19-dev in MongoDB. Time: --- 0.39432287216186523 secnds ---
Script execution completed at 21/04/2020 20:44:44. Time: --- 0.06860756874084473 secnds ---
Dataframe FRANCE_DB_CONSO successfully save in database covid-19-dev in MongoDB. Time: --- 0.5067164897918701 secnds ---
Script execution completed at 21/04/2020 20:44:44. Time: --- 0.22898125648498535 secnds ---
Dataframe FRANCE_DB_ALL successfully save in database covid-19-dev in MongoDB. Time: --- 12.391313076019287 secnds ---
Script execution completed at 21/04/2020 20:44:44. Time: --- 0.08114361763000488 secnds ---
Dataframe FRANCE_DB_TREND successfully save in database covid-19-dev in MongoDB. Time: --- 11.94045901298523 secnds ---


In [4]:
#--- Load data source
start_time = time.time()
db_all_fr = get_datasource('FRANCE_DB_ALL')
db_trend_fr = get_datasource('FRANCE_DB_TREND')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 17/04/2020 15:43:52. Time: --- 0.14144134521484375 secnds ---


In [5]:
#-- Covid-19 - DATA.GOUV.FR TENDANCES
start_time = time.time()
def domain_105(df,domain_num):
    #-- Create Datamodel
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'GROUPS': domain['STATUS_NAME'],
        'GROUPS_ORDER': domain['STATUS_ORDER'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'BOTTOM_FILTER': 'Depuis le début',
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': domain['LABEL'],
        'LABEL_GROUPS': domain['LABEL_GROUPS'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
    })
    
    #-- Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARV','UPPER_FILTER_M'] = 'VARIATION EN NB'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARP','UPPER_FILTER_M'] = 'VARIATION EN %'
    
    #-- Create new date scenario in BOTTOM_FILTER
    df1 = get_lastdays(domain, 15,"15 derniers jours",'BOTTOM_FILTER')
    domain = pd.concat([domain,df1],axis=0)
    
    #-- Filter upperfilter value to get France and detailed by region
    domain_dep = domain[domain['LABEL_GROUPS'] != 'France']
    domain_fra = domain[domain['UPPER_FILTER_R'] == 'France']
    domain = pd.concat([domain_dep,domain_fra],axis=0)
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    
    #-- Create upper filter domain
    upper_filter = domain[['UPPER_FILTER_R']].drop_duplicates().sort_values(by='UPPER_FILTER_R', ascending=True).reset_index(drop=True)
    upper_filter['ORDER'] = upper_filter.index + 1
    upper_filter.loc[upper_filter['UPPER_FILTER_R'] == 'France', 'ORDER'] = 0
    upper_filter = upper_filter.sort_values(by='ORDER',ascending=True)
    upper_filter.to_csv(output_folder + '105_UPPER_FILTER.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(upper_filter,'105_UPPER_FILTER',db_app,True)
    return domain

domain105 = domain_105(db_trend_fr,'105')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain105

Dataframe 105 successfully save in database app-wsr-dev in MongoDB. Time: --- 18.07766890525818 secnds ---
Dataframe 105_UPPER_FILTER successfully save in database app-wsr-dev in MongoDB. Time: --- 0.025971412658691406 secnds ---
Script execution completed at 17/04/2020 15:44:11. Time: --- 18.812723636627197 secnds ---


,ENTITY,SCENARIO,GROUPS,GROUPS_ORDER,DATE_ORDER,BOTTOM_FILTER,UPPER_FILTER_M,UPPER_FILTER_R,LABEL_GROUPS,VALUE,UNIT_VALUE,PRECISION,LAST_UPDATE
0,France,16/04/2020,Hospitalisations,1.0,20200416,Depuis le début,EVOLUTION,31 - Haute-Garonne,Occitanie,212.000000,cas,",.0f",17/04/2020 15:43:53
1,France,16/04/2020,Hospitalisations,1.0,20200416,Depuis le début,EVOLUTION,32 - Gers,Occitanie,32.000000,cas,",.0f",17/04/2020 15:43:53
2,France,16/04/2020,Hospitalisations,1.0,20200416,Depuis le début,EVOLUTION,33 - Gironde,Nouvelle-Aquitaine,296.000000,cas,",.0f",17/04/2020 15:43:53
3,France,16/04/2020,Hospitalisations,1.0,20200416,Depuis le début,EVOLUTION,34 - Hérault,Occitanie,161.000000,cas,",.0f",17/04/2020 15:43:53
4,France,16/04/2020,Hospitalisations,1.0,20200416,Depuis le début,EVOLUTION,35 - Ille-et-Vilaine,Bretagne,193.000000,cas,",.0f",17/04/2020 15:43:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,France,02/04/2020,Retours à domicile,3.0,20200402,15 derniers jours,VARIATION EN NB,France,France,1493.000000,cas,"+,.0f",17/04/2020 15:43:53
53852,France,02/04/2020,Hospitalisations,1.0,20200402,15 derniers jours,VARIATION EN %,France,France,6.470277,%,"+,.2f",17/04/2020 15:43:53
53853,France,02/04/2020,Décès à l’hôpital,4.0,20200402,15 derniers jours,VARIATION EN %,France,France,11.681548,%,"+,.2f",17/04/2020 15:43:53
53854,France,02/04/2020,En réanimation,2.0,20200402,15 derniers jours,VARIATION EN %,France,France,6.144781,%,"+,.2f",17/04/2020 15:43:53


In [6]:
#-- Covid-19 - DATA.GOUV.FR CLASSEMENT
start_time = time.time()
def domain_107(df,domain_num):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'CHILD': domain['LABEL'],
        'PARENT': domain['LABEL_GROUPS'],
        'UPPER_FILTER_M': domain['STATUS_NAME'],
        'ORDER':domain['STATUS_ORDER'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VARP': ' %',
        'PRECISION_VALUE': ',.0f',
        'PRECISION_VAR': '+,.0f',
        'PRECISION_VARP': '+,.2f',
    })
    
    domain.loc[domain['PARENT'] == 'France', 'PARENT'] = 'root'
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    return domain

domain107 = domain_107(db_all_fr,'107')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain107

Dataframe 107 successfully save in database app-wsr-dev in MongoDB. Time: --- 11.841386556625366 secnds ---
Script execution completed at 17/04/2020 15:44:23. Time: --- 11.988147735595703 secnds ---


,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,UNIT_VARP,PRECISION_VALUE,PRECISION_VAR,PRECISION_VARP,LAST_UPDATE
0,France,16/04/2020,20200416,Provence-Alpes-Côte d'Azur,root,En réanimation,2.0,393,-10,-2.481390,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
1,France,16/04/2020,20200416,Provence-Alpes-Côte d'Azur,root,Hospitalisations,1.0,1883,-26,-1.361970,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
2,France,16/04/2020,20200416,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,212,-10,-4.504505,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
3,France,16/04/2020,20200416,32 - Gers,Occitanie,Hospitalisations,1.0,32,1,3.225806,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
4,France,16/04/2020,20200416,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,296,-8,-2.631579,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,France,19/03/2020,20200319,87 - Haute-Vienne,Nouvelle-Aquitaine,Retours à domicile,3.0,2,0,0.000000,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
13916,France,19/03/2020,20200319,80 - Somme,Hauts-de-France,Retours à domicile,3.0,12,3,33.333333,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
13917,France,19/03/2020,20200319,79 - Deux-Sèvres,Nouvelle-Aquitaine,Retours à domicile,3.0,3,1,50.000000,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11
13918,France,19/03/2020,20200319,78 - Yvelines,Ile-de-France,Retours à domicile,3.0,7,2,40.000000,%,",.0f","+,.0f","+,.2f",17/04/2020 15:44:11


In [5]:
if use_healthchecks:
    step_2 = hc.done(hc_key)
    print('End ==>', step_2)

End ==> End ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-04-22
